In [ ]:
# Run setup from config notebook
%run 0_config_setup.ipynb

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
from datetime import datetime
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

set_seed(SEED)

## Load Models

In [ ]:
print("Loading models...\n")

# Load PPO-optimized translation model
print(f"Loading translation model from {PPO_MODEL_COLD_START}...")
translation_tokenizer = AutoTokenizer.from_pretrained(PPO_MODEL_COLD_START)
if translation_tokenizer.pad_token is None:
    translation_tokenizer.pad_token = translation_tokenizer.eos_token

translation_model = AutoModelForCausalLM.from_pretrained(
    PPO_MODEL_COLD_START,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)
translation_model.eval()
print("✓ Translation model loaded")

# Load reward model
print(f"\nLoading reward model from {REWARD_MODEL_COLD_START}...")
rm_tokenizer = AutoTokenizer.from_pretrained(REWARD_MODEL_COLD_START)

rm_base_model = AutoModelForCausalLM.from_pretrained(
    REWARD_BASE_MODEL,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

# Recreate reward model
import torch.nn as nn

class RewardModel(nn.Module):
    def __init__(self, base_model, hidden_dim=256, head_type='mlp'):
        super().__init__()
        self.base_model = base_model
        self.head_type = head_type
        self.hidden_size = base_model.config.hidden_size
        
        if head_type == 'linear':
            self.reward_head = nn.Linear(self.hidden_size, 1)
        elif head_type == 'mlp':
            self.reward_head = nn.Sequential(
                nn.Linear(self.hidden_size, hidden_dim),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(hidden_dim, 1)
            )
    
    def forward(self, input_ids, attention_mask):
        outputs = self.base_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )
        hidden_states = outputs.hidden_states[-1]
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = hidden_states.shape[0]
        pooled = hidden_states[torch.arange(batch_size), sequence_lengths]
        reward = self.reward_head(pooled)
        return reward.squeeze(-1)

reward_model = RewardModel(
    base_model=rm_base_model,
    hidden_dim=RM_HIDDEN_DIM,
    head_type=RM_HEAD_TYPE
)

checkpoint = torch.load(
    REWARD_MODEL_COLD_START / "reward_model.pt",
    map_location='cpu'
)
reward_model.load_state_dict(checkpoint['model_state_dict'])
reward_model.eval()

print("✓ Reward model loaded")
print("\nAll models ready!")

## Translation Pipeline

In [ ]:
def generate_candidates(source_text, source_lang, num_candidates=8):
    """Generate multiple translation candidates"""
    
    prompt = format_translation_prompt(source_text, source_lang)
    inputs = translation_tokenizer(prompt, return_tensors="pt").to(translation_model.device)
    
    candidates = []
    
    # Generate with different sampling parameters
    for i in range(num_candidates):
        # Vary temperature and sampling
        temp = 0.7 + (i * 0.1)  # 0.7 to 1.4
        top_p = 0.85 + (i * 0.02)  # 0.85 to 0.99
        
        with torch.no_grad():
            outputs = translation_model.generate(
                **inputs,
                max_new_tokens=INFERENCE_MAX_LENGTH,
                temperature=min(temp, 1.5),
                top_p=min(top_p, 0.99),
                top_k=50,
                do_sample=True,
                num_return_sequences=1,
                pad_token_id=translation_tokenizer.pad_token_id,
                eos_token_id=translation_tokenizer.eos_token_id
            )
        
        full_text = translation_tokenizer.decode(outputs[0], skip_special_tokens=True)
        translation = full_text.split("Arabic translation:")[-1].strip()
        
        candidates.append(translation)
    
    # Remove duplicates while preserving order
    seen = set()
    unique_candidates = []
    for cand in candidates:
        if cand not in seen:
            seen.add(cand)
            unique_candidates.append(cand)
    
    return unique_candidates


def score_candidates(source_text, candidates):
    """Score candidates using the reward model"""
    
    scored_candidates = []
    
    for translation in candidates:
        text = f"Source: {source_text}\nTranslation: {translation}"
        
        inputs = rm_tokenizer(
            text,
            max_length=RM_MAX_LENGTH,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        ).to(reward_model.base_model.device)
        
        with torch.no_grad():
            reward = reward_model(
                inputs['input_ids'],
                inputs['attention_mask']
            ).item()
        
        scored_candidates.append({
            'translation': translation,
            'score': reward
        })
    
    # Sort by score (descending)
    scored_candidates.sort(key=lambda x: x['score'], reverse=True)
    
    return scored_candidates


def translate_with_ranking(source_text, source_lang='en'):
    """Complete translation pipeline with ranking"""
    
    print(f"Generating {INFERENCE_NUM_CANDIDATES} candidate translations...")
    candidates = generate_candidates(source_text, source_lang, INFERENCE_NUM_CANDIDATES)
    print(f"✓ Generated {len(candidates)} unique candidates")
    
    print("\nScoring candidates with reward model...")
    scored = score_candidates(source_text, candidates)
    print("✓ Scoring complete")
    
    # Get top K
    top_candidates = scored[:INFERENCE_TOP_K]
    
    return top_candidates, scored

print("Translation pipeline ready!")

## Feedback Storage

In [ ]:
def save_feedback(source_text, source_lang, candidates, user_ranking, custom_translation=None):
    """Save user feedback to file"""
    
    feedback_entry = {
        'timestamp': datetime.now().isoformat(),
        'source': source_text,
        'source_lang': source_lang,
        'candidates': candidates,
        'user_ranking': user_ranking,
        'custom_translation': custom_translation
    }
    
    # Append to feedback file
    with open(HUMAN_PREFERENCES, 'a', encoding='utf-8') as f:
        f.write(json.dumps(feedback_entry, ensure_ascii=False) + '\n')
    
    print(f"✓ Feedback saved to {HUMAN_PREFERENCES}")

print("Feedback storage ready!")

## Interactive Translation Interface

In [ ]:
# Create interface
print("Creating interactive interface...\n")

# Input widgets
source_input = widgets.Textarea(
    value='',
    placeholder='Enter text to translate...',
    description='Source:',
    layout=widgets.Layout(width='80%', height='100px')
)

lang_select = widgets.Dropdown(
    options=[('English', 'en'), ('French', 'fr')],
    value='en',
    description='Language:',
)

translate_button = widgets.Button(
    description='Translate',
    button_style='primary',
    icon='language'
)

output_area = widgets.Output()

# Storage for current session
current_session = {
    'source': None,
    'lang': None,
    'candidates': None,
    'all_candidates': None
}

def on_translate_click(b):
    with output_area:
        clear_output(wait=True)
        
        source_text = source_input.value.strip()
        if not source_text:
            print("Please enter text to translate.")
            return
        
        source_lang = lang_select.value
        
        print("=" * 80)
        print(f"Source ({lang_select.options[lang_select.index][0]}): {source_text}")
        print("=" * 80)
        print()
        
        # Generate and rank translations
        top_candidates, all_candidates = translate_with_ranking(source_text, source_lang)
        
        # Store in session
        current_session['source'] = source_text
        current_session['lang'] = source_lang
        current_session['candidates'] = top_candidates
        current_session['all_candidates'] = all_candidates
        
        # Display top candidates
        print("\n" + "=" * 80)
        print(f"Top {len(top_candidates)} Translations (ranked by quality):")
        print("=" * 80)
        
        for i, cand in enumerate(top_candidates, 1):
            print(f"\n{i}. {cand['translation']}")
            print(f"   Score: {cand['score']:.4f}")
        
        print("\n" + "=" * 80)
        print("\nTo provide feedback, use the feedback functions below.")

translate_button.on_click(on_translate_click)

# Display interface
display(widgets.VBox([
    widgets.HTML("<h2>🌐 Arabic Translation System</h2>"),
    source_input,
    lang_select,
    translate_button,
    output_area
]))

print("Interface ready! Use the form above to translate.")

## Provide Feedback

After seeing translations, provide feedback using the cells below.

In [ ]:
# Option 1: Rank the displayed translations
# Provide ranking as list of indices (1, 2, 3)
# Example: [2, 1, 3] means "translation 2 is best, then 1, then 3"

def submit_ranking(ranking):
    """
    Submit ranking for current translations.
    
    Args:
        ranking: List of indices (1-based) in order of preference
                 Example: [2, 1, 3] means option 2 is best
    """
    if current_session['candidates'] is None:
        print("No translations to rank. Please translate text first.")
        return
    
    print(f"Submitting ranking: {ranking}")
    
    # Convert to 0-based and validate
    try:
        ranking_0based = [r - 1 for r in ranking]
        if not all(0 <= r < len(current_session['candidates']) for r in ranking_0based):
            print("Invalid ranking indices!")
            return
    except:
        print("Invalid ranking format!")
        return
    
    save_feedback(
        current_session['source'],
        current_session['lang'],
        current_session['candidates'],
        ranking,
        custom_translation=None
    )
    
    print("Thank you for your feedback!")

# Example usage:
# submit_ranking([2, 1, 3])  # Translation 2 is best, then 1, then 3

In [ ]:
# Option 2: Provide your own translation

def submit_custom_translation(custom_text, best_candidate_idx=None):
    """
    Submit a custom translation.
    
    Args:
        custom_text: Your custom Arabic translation
        best_candidate_idx: (Optional) Index of best system translation (1-based)
    """
    if current_session['candidates'] is None:
        print("No session active. Please translate text first.")
        return
    
    print(f"Submitting custom translation: {custom_text[:100]}...")
    
    ranking = [best_candidate_idx] if best_candidate_idx else []
    
    save_feedback(
        current_session['source'],
        current_session['lang'],
        current_session['candidates'],
        ranking,
        custom_translation=custom_text
    )
    
    print("Thank you for your feedback!")

# Example usage:
# submit_custom_translation("مرحبا، كيف حالك؟", best_candidate_idx=1)

## Batch Translation (Optional)

Translate multiple texts at once for efficiency.

In [ ]:
def batch_translate(texts, source_lang='en'):
    """Translate multiple texts"""
    
    results = []
    
    for i, text in enumerate(texts, 1):
        print(f"\nTranslating {i}/{len(texts)}: {text[:50]}...")
        
        top_candidates, _ = translate_with_ranking(text, source_lang)
        
        results.append({
            'source': text,
            'best_translation': top_candidates[0]['translation'],
            'score': top_candidates[0]['score'],
            'all_top_candidates': top_candidates
        })
        
        print(f"Best: {top_candidates[0]['translation'][:80]}...")
    
    return results

# Example usage:
# batch_texts = [
#     "Hello, how are you?",
#     "The weather is nice today.",
#     "I love learning new languages."
# ]
# batch_results = batch_translate(batch_texts, source_lang='en')

## View Collected Feedback

In [ ]:
# Check how much feedback has been collected
try:
    with open(HUMAN_PREFERENCES, 'r', encoding='utf-8') as f:
        feedback_count = sum(1 for line in f)
    print(f"Total feedback entries collected: {feedback_count}")
    
    if feedback_count > 0:
        print("\nSample feedback entries:")
        with open(HUMAN_PREFERENCES, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i >= 3:  # Show first 3
                    break
                entry = json.loads(line)
                print(f"\n{i+1}. Source: {entry['source'][:60]}...")
                print(f"   Ranking: {entry.get('user_ranking', 'N/A')}")
                print(f"   Custom: {entry.get('custom_translation', 'None')[:60] if entry.get('custom_translation') else 'None'}...")
except FileNotFoundError:
    print("No feedback collected yet.")

## Next Step

Once you have collected sufficient human feedback (recommended: 500+ preference pairs), proceed to **notebook 5** to fine-tune the reward model with human preferences and re-run PPO for final alignment.